In [ ]:
!pip install ultralytics
from ultralytics import YOLO
model = YOLO("yolo11n.pt")
print("Pretrained YOLOv11n model downloaded and loaded successfully!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.2/922.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 44.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

100%|██████████| 5.35M/5.35M [00:00<00:00, 63.4MB/s]

Pretrained YOLOv11n model downloaded and loaded successfully!


In [ ]:
!pip install xmltodict  # For parsing XML annotations
import os
import shutil
import xmltodict
from ultralytics import YOLO
import yaml
from google.colab import drive

In [ ]:
drive.mount('/content/drive')
dataset_paths = {
    "Missing_hole": {
        "images": "/content/drive/My Drive/PCB/PCB_DATASET/images/Missing_hole",
        "annotations": "/content/drive/My Drive/PCB/PCB_DATASET/Annotations/Missing_hole"
    },
    "Mouse_bite": {
        "images": "/content/drive/My Drive/PCB/PCB_DATASET/images/Mouse_bite",
        "annotations": "/content/drive/My Drive/PCB/PCB_DATASET/Annotations/Mouse_bite"
    },
    "Open_circuit": {
        "images": "/content/drive/My Drive/PCB/PCB_DATASET/images/Open_circuit",
        "annotations": "/content/drive/My Drive/PCB/PCB_DATASET/Annotations/Open_circuit"
    },
    "Short": {
        "images": "/content/drive/My Drive/PCB/PCB_DATASET/images/Short",
        "annotations": "/content/drive/My Drive/PCB/PCB_DATASET/Annotations/Short"
    },
    "Spurious_copper": {
        "images": "/content/drive/My Drive/PCB/PCB_DATASET/images/Spurious_copper",
        "annotations": "/content/drive/My Drive/PCB/PCB_DATASET/Annotations/Spurious_copper"
    },
    "Spur": {
        "images": "/content/drive/My Drive/PCB/PCB_DATASET/images/Spur",
        "annotations": "/content/drive/My Drive/PCB/PCB_DATASET/Annotations/Spur"
    }
}
class_names = ["missing_hole", "mouse_bite", "open_circuit", "short", "spurious_copper", "spur"]
num_classes = len(class_names)

Mounted at /content/drive


In [ ]:
base_dir = "/content/pcb_dataset_yolo"
os.makedirs(f"{base_dir}/images/train", exist_ok=True)
os.makedirs(f"{base_dir}/images/val", exist_ok=True)
os.makedirs(f"{base_dir}/labels/train", exist_ok=True)
os.makedirs(f"{base_dir}/labels/val", exist_ok=True)

In [ ]:
# Step 6: Convert PASCAL VOC to YOLO format
def convert_voc_to_yolo(xml_file, img_width, img_height, class_names):
    with open(xml_file, 'r') as f:
        xml_dict = xmltodict.parse(f.read())['annotation']

    labels = []
    if 'object' in xml_dict:
        objects = xml_dict['object']
        if not isinstance(objects, list):
            objects = [objects]

        for obj in objects:
            class_name = obj['name']
            class_id = class_names.index(class_name)

            # Get bounding box coordinates
            bbox = obj['bndbox']
            xmin, ymin, xmax, ymax = map(float, [bbox['xmin'], bbox['ymin'], bbox['xmax'], bbox['ymax']])

            # Convert to YOLO format (normalized center x, center y, width, height)
            x_center = (xmin + xmax) / 2 / img_width
            y_center = (ymin + ymax) / 2 / img_height
            width = (xmax - xmin) / img_width
            height = (ymax - ymin) / img_height

            labels.append(f"{class_id} {x_center} {y_center} {width} {height}")

    return labels

In [ ]:
# Step 7: Organize dataset and split into train/val
for defect, paths in dataset_paths.items():
    image_dir = paths["images"]
    annotation_dir = paths["annotations"]

    images = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png'))]
    split_idx = int(0.8 * len(images))  # 80% train, 20% val

    for i, img_file in enumerate(images):
        img_path = os.path.join(image_dir, img_file)
        xml_file = os.path.join(annotation_dir, img_file.replace('.jpg', '.xml').replace('.png', '.xml'))

        # Get image dimensions
        from PIL import Image
        img = Image.open(img_path)
        img_width, img_height = img.size

        # Convert annotations
        if os.path.exists(xml_file):
            labels = convert_voc_to_yolo(xml_file, img_width, img_height, class_names)
            label_file = os.path.join(base_dir, "labels", "train" if i < split_idx else "val", img_file.replace('.jpg', '.txt').replace('.png', '.txt'))
            with open(label_file, 'w') as f:
                f.write("\n".join(labels))

        # Copy image
        dest_dir = "train" if i < split_idx else "val"
        shutil.copy(img_path, os.path.join(base_dir, "images", dest_dir, img_file))

# Step 8: Create YAML configuration file
data_yaml = {
    "train": f"{base_dir}/images/train",
    "val": f"{base_dir}/images/val",
    "nc": num_classes,
    "names": class_names
}
with open(f"{base_dir}/data.yaml", 'w') as f:
    yaml.dump(data_yaml, f)

In [ ]:
# Step 9: Load and fine-tune YOLOv11n
results = model.train(data=f"{base_dir}/data.yaml",epochs=50,batch=8,imgsz=640,iou=0.5,device='cpu',patience=10)    #device='0' for gpu
# Step 10: Evaluate the model (mAP)
metrics = model.val()
mAP_50 = metrics.box.map50  # mAP@0.5
mAP_50_95 = metrics.box.map  # mAP@0.5:0.95
print(f"mAP@0.5: {mAP_50:.4f}")
print(f"mAP@0.5:0.95: {mAP_50_95:.4f}")

Ultralytics 8.3.83 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (AMD EPYC 7B12)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/content/pcb_dataset_yolo/data.yaml, epochs=50, time=None, patience=10, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.5, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=T

100%|██████████| 755k/755k [00:00<00:00, 14.3MB/s]


Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/pcb_dataset_yolo/labels/train... 552 images, 0 backgrounds, 0 corrupt: 100%|██████████| 552/552 [00:00<00:00, 1680.08it/s]

train: New cache created: /content/pcb_dataset_yolo/labels/train.cache


...


46 epochs completed in 4.245 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 5.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.83 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (AMD EPYC 7B12)
YOLO11n summary (fused): 100 layers, 2,583,322 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:31<00:00,  3.51s/it]


                   all        141        712      0.899      0.872      0.914      0.471
          missing_hole         23        119      0.991      0.958      0.991      0.593
            mouse_bite         23        115      0.817      0.843      0.872      0.438
          open_circuit         24        121      0.949      0.769      0.869      0.397
                 short         24        120      0.971      0.967      0.982      0.501
       spurious_copper         24        121      0.823      0.925       0.93      0.489
                  spur         23        116      0.845      0.767      0.841      0.408
Speed: 3.9ms preprocess, 178.2ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs/detect/train
Ultralytics 8.3.83 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (AMD EPYC 7B12)
YOLO11n summary (fused): 100 layers, 2,583,322 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning /content/pcb_dataset_yolo/labels/val.cache... 141 images, 0 backgrounds, 0 corrupt: 100%|██████████| 141/141 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:27<00:00,  1.50s/it]


                   all        141        712      0.899      0.872      0.914      0.471
          missing_hole         23        119      0.991      0.958      0.991      0.593
            mouse_bite         23        115      0.817      0.843      0.872      0.438
          open_circuit         24        121      0.949      0.769      0.869      0.397
                 short         24        120      0.971      0.967      0.982      0.501
       spurious_copper         24        121      0.823      0.925       0.93      0.489
                  spur         23        116      0.845      0.767      0.841      0.408
Speed: 0.9ms preprocess, 149.8ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs/detect/train2
mAP@0.5: 0.9142
mAP@0.5:0.95: 0.4710


In [ ]:
# Step 10: Evaluate the model (mAP)
metrics = model.val()
mAP_50 = metrics.box.map50  # mAP@0.5
mAP_50_95 = metrics.box.map  # mAP@0.5:0.95
print(f"mAP@0.5: {mAP_50:.4f}")
print(f"mAP@0.5:0.95: {mAP_50_95:.4f}")

Ultralytics 8.3.83 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (AMD EPYC 7B12)


val: Scanning /content/pcb_dataset_yolo/labels/val.cache... 141 images, 0 backgrounds, 0 corrupt: 100%|██████████| 141/141 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:32<00:00,  1.82s/it]


                   all        141        712      0.899      0.872      0.914      0.471
          missing_hole         23        119      0.991      0.958      0.991      0.593
            mouse_bite         23        115      0.817      0.843      0.872      0.438
          open_circuit         24        121      0.949      0.769      0.869      0.397
                 short         24        120      0.971      0.967      0.982      0.501
       spurious_copper         24        121      0.823      0.925       0.93      0.489
                  spur         23        116      0.845      0.767      0.841      0.408
Speed: 1.0ms preprocess, 184.2ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train3
mAP@0.5: 0.9142
mAP@0.5:0.95: 0.4710


In [ ]:
confusion_matrix = metrics.confusion_matrix.matrix
print("\nConfusion Matrix:")
print(confusion_matrix)


Confusion Matrix:
[[        113           0           0           0           0           0           1]
 [          0          99          10           0           1           4           6]
 [          0           2          91           0           0           0           5]
 [          0           1           0         116           0           0           2]
 [          0           1           2           0         111          15           8]
 [          0           0           5           0           1          82          17]
 [          6          12          13           4           8          15           0]]


In [ ]:
model.save("/content/drive/My Drive/PCB/yolo11n_pcb_trained.pt")
print("Model saved to /content/drive/My Drive/PCB/yolo11n_pcb_trained.pt")

Model saved to /content/drive/My Drive/PCB/yolo11n_pcb_trained.pt
